In [1]:
import os
import sys
import operator
import numpy as np
import pandas as pd
import scipy
import dateutil
from sklearn.model_selection import train_test_split,KFold
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report, f1_score
from sklearn import tree
import seaborn as sns
import matplotlib.pyplot as plt
import csv
from functools import reduce

In [2]:
# df_paths_lists = [r"features.csv", r"binary_features_users.csv","number_sessions.csv","feature_hist.csv"]
# df_paths_lists = [r"features.csv", r"binary_features_users.csv"]
df_paths_lists = ["feature_hist.csv"]
df_list = [pd.read_csv(path) for path in df_paths_lists]
for dataframe in df_list:
    print(list(dataframe))
df = reduce(lambda df1, df2: df1.merge(df2, "outer"), df_list)
df.dropna(inplace=True)
# df = pd.read_csv(r"features.csv")
df.head()

['id', 'Adjust', 'Backdrop', 'Crop', 'Darkroom', 'Defocus', 'Eyes', 'Face', 'Filters', 'Light FX', 'Paint', 'Patch', 'Prism', 'Relight', 'Reshape', 'Retouch', 'Touch Up', 'Vignette', 'accepted', 'churned']


,id,Adjust,Backdrop,Crop,Darkroom,Defocus,Eyes,Face,Filters,Light FX,Paint,Patch,Prism,Relight,Reshape,Retouch,Touch Up,Vignette,accepted,churned
0,20000443246052,1,0,14,2,1,1,0,0,17,6,0,2,3,5,10,3,25,0.600000,0
1,20000443252617,2,2,0,7,0,0,0,0,16,1,0,10,2,0,4,3,14,0.409836,0
2,20000443310150,2,0,0,9,0,3,1,0,3,1,0,3,0,0,6,0,3,0.290323,0
3,20000443367561,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1.000000,0
4,20000443405819,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,3,0.500000,0


In [3]:
# df = df.set_index(df["id"])
df.head()

,id,Adjust,Backdrop,Crop,Darkroom,Defocus,Eyes,Face,Filters,Light FX,Paint,Patch,Prism,Relight,Reshape,Retouch,Touch Up,Vignette,accepted,churned
0,20000443246052,1,0,14,2,1,1,0,0,17,6,0,2,3,5,10,3,25,0.600000,0
1,20000443252617,2,2,0,7,0,0,0,0,16,1,0,10,2,0,4,3,14,0.409836,0
2,20000443310150,2,0,0,9,0,3,1,0,3,1,0,3,0,0,6,0,3,0.290323,0
3,20000443367561,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1.000000,0
4,20000443405819,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,3,0.500000,0


In [4]:
features_names = list(df)
features_names.remove("id")
features_names.remove("churned")

In [5]:
features_names

['Adjust',
 'Backdrop',
 'Crop',
 'Darkroom',
 'Defocus',
 'Eyes',
 'Face',
 'Filters',
 'Light FX',
 'Paint',
 'Patch',
 'Prism',
 'Relight',
 'Reshape',
 'Retouch',
 'Touch Up',
 'Vignette',
 'accepted']

In [17]:
len(features_names)

18

In [6]:
# # features_names = ["example_feature","example_numeric"]
# features_names = "num_usages"
features = df[features_names] # can add more features
labels = df["churned"]
X_train, X_val, y_train, y_val = train_test_split(features, labels, test_size=0.2)
X_train = X_train.values#.reshape(-1,1)
X_val = X_val.values#.reshape(-1,1)
y_train = y_train.values
y_val = y_val.values

In [16]:
features

,Adjust,Backdrop,Crop,Darkroom,Defocus,Eyes,Face,Filters,Light FX,Paint,Patch,Prism,Relight,Reshape,Retouch,Touch Up,Vignette,accepted
0,1,0,14,2,1,1,0,0,17,6,0,2,3,5,10,3,25,0.600000
1,2,2,0,7,0,0,0,0,16,1,0,10,2,0,4,3,14,0.409836
2,2,0,0,9,0,3,1,0,3,1,0,3,0,0,6,0,3,0.290323
3,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1.000000
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,3,0.500000
5,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,3,0.750000
6,12,7,7,2,0,6,4,1,21,4,0,0,4,7,7,7,11,0.700000
7,0,0,2,0,1,0,2,0,27,1,0,11,0,0,3,7,11,0.507692
8,1,0,0,0,0,0,1,1,4,0,0,1,0,0,0,3,9,0.600000
9,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,0.333333


In [8]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [9]:
from sklearn.ensemble import RandomForestClassifier
# Use the random grid to search for best hyperparameters
# First create the base model to tune


rf = RandomForestClassifier()
# Random search of parametrs, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1, scoring = "f1")
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   41.1s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  6.7min finished


RandomizedSearchCV(cv=3, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid=True, n_iter=100, n_jobs=-1,
          param_distributions={'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring='f1', verbose=2)

In [10]:
rf_random.best_params_


{'n_estimators': 1400,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': 40,
 'bootstrap': False}

In [11]:
random_grid

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000],
 'max_features': ['auto', 'sqrt'],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'min_samples_split': [2, 5, 10],
 'min_samples_leaf': [1, 2, 4],
 'bootstrap': [True, False]}

In [12]:
import sklearn
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    f1 = sklearn.metrics.f1_score(test_labels,predictions)
    return f1
base_model = RandomForestClassifier(n_estimators = 10, random_state = 42)
base_model.fit(X_train, y_train)
base_f1 = evaluate(base_model, X_val, y_val)
print(base_f1)

0.03947368421052631


In [13]:
best_random = rf_random.best_estimator_
best_f1_train = evaluate(best_random, X_train, y_train)
print(best_f1_train)
best_f1_val = evaluate(best_random, X_val, y_val)
print(best_f1_val)

0.9550359712230215
0.0


In [14]:
best_f = evaluate(best_random, X_val, y_val)
print(best_f1)

NameError: name 'best_f1' is not defined

In [ ]:
test_df = pd.read_csv(r"LightricksSource\test_users_data.csv")
# test_df = test_df.set_index(test_df["id"])
# test_df["example_feature"] = test_df["country"] == "United States"
# test_df["example_numeric"] = np.random.randint(1, 6, test_df.shape[0])
X_test = test_df[features_names]
y_test = best_random.predict(X_test)
test_df["churned"] = y_test
test_df = test_df.rename(index=str, columns={"Unnamed: 0": ""})
test_df.to_csv("submission.csv",index=False, quoting=1)

In [ ]:
test_df